## 3. Customer segmentation by Gaussian Mixture Model

**미완성입니다* - 필요는 없을거같습니다.

In [ ]:
import os
print(os.getcwd())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import hashlib

df = pd.read_csv('all_customers_shopify_1-13.csv', delimiter=',')

df.head()

## 1. Data preview and cleaning

In [ ]:
df_original = df.copy(deep=True)

df.head()

In [ ]:
df = df[['Email', 'Total Spent', 'Total Orders']]

In [ ]:
df.head()

In [ ]:
df = df[df['Total Spent'] > 0]

In [ ]:
df

In [ ]:
df['AvgOrderValue'] = df['Total Spent'] / df['Total Orders']
df.head()

In [ ]:
df.describe()

In [ ]:
df.head()

## 2. Normalize the values

#### to feed into our Cluster Analysis ML Model for training

In [ ]:
df

In [ ]:
df_normalized = (df - df.mean()) / df.std()

In [ ]:
df_normalized

In [ ]:
df_normalized['Email'] = df['Email']
columns_titles = ["Email", "Total Orders", "Total Spent", "AvgOrderValue"]
df_normalized = df_normalized.reindex(columns=columns_titles)
df_normalized

In [ ]:
df_normalized.describe()

## 3. Customer segmentation by Gaussian Mixture Model

In [ ]:
from sklearn.mixture import GaussianMixture

gaussian = GaussianMixture(n_components = 3, n_init = 10)
gaussian.fit(df_normalized[["Total Orders", "Total Spent", "AvgOrderValue"]])

In [ ]:
gaussian.weights_

In [ ]:
gaussian.means_

In [ ]:
gaussian.covariances_

In [ ]:
gaussian.converged_

In [ ]:
gaussian.n_iter_

In [ ]:
gaussian.predict(df_normalized[["Total Orders", "Total Spent", "AvgOrderValue"]])

In [ ]:
probs = gaussian.predict_proba(df_normalized[["Total Orders", "Total Spent", "AvgOrderValue"]])

In [ ]:
x_new, y_new = gaussian.sample(6)
x_new

In [ ]:
y_new

In [ ]:
gaussian.score_samples(df_normalized[["Total Orders", "Total Spent", "AvgOrderValue"]])

In [ ]:
X = df_normalized[["Total Orders", "Total Spent", "AvgOrderValue"]]

from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()

    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)

    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))


In [ ]:
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X['Total Orders'], X['Total Spent'], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X['Total Orders'], X['Total Spent'], s=40, zorder=2)
    ax.axis('equal')

    w_factor = 0.2 / gmm.weights_.max()
    # for pos, covar, w in zip(gmm.means_, gmm.covars_, gmm.weights_):
    #     draw_ellipse(pos, covar, alpha=w * w_factor)

    for pos, covar, w in zip(gmm.means_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)
